![logog](https://raw.githubusercontent.com/Pacific-AI-Corp/langtest/main/docs/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Pacific-AI-Corp/langtest/blob/main/demo/tutorials/misc/Templatic_Augmentation_Notebook.ipynb)

**LangTest** is an open-source python library designed to help developers deliver safe and effective Natural Language Processing (NLP) models. Whether you are using **John Snow Labs, Hugging Face, Spacy** models or **OpenAI, Cohere, AI21, Hugging Face Inference API and Azure-OpenAI** based LLMs, it has got you covered. You can test any Named Entity Recognition (NER), Text Classification, fill-mask, Translation model using the library. We also support testing LLMS for Question-Answering, Summarization and text-generation tasks on benchmark datasets. The library supports 60+ out of the box tests. For a complete list of supported test categories, please refer to the [documentation](http://langtest.org/docs/pages/docs/test_categories).

Metrics are calculated by comparing the model's extractions in the original list of sentences against the extractions carried out in the noisy list of sentences. The original annotated labels are not used at any point, we are simply comparing the model against itself in a 2 settings.

# Getting started with LangTest on John Snow Labs

In [ ]:
!pip install "langtest[johnsnowlabs,openai]"

# Harness and its Parameters

The Harness class is a testing class for Natural Language Processing (NLP) models. It evaluates the performance of a NLP model on a given task using test data and generates a report with test results.Harness can be imported from the LangTest library in the following way.

In [2]:
#Import Harness from the LangTest library
from langtest import Harness

It imports the Harness class from within the module, that is designed to provide a blueprint or framework for conducting NLP testing, and that instances of the Harness class can be customized or configured for different testing scenarios or environments.

Here is a list of the different parameters that can be passed to the Harness function:

<br/>



| Parameter     | Description |
| - | - |
| **task**      | Task for which the model is to be evaluated (text-classification or ner) |
| **model**     | Specifies the model(s) to be evaluated. This parameter can be provided as either a dictionary or a list of dictionaries. Each dictionary should contain the following keys: <ul><li>model (mandatory): 	PipelineModel or path to a saved model or pretrained pipeline/model from hub.</li><li>hub (mandatory): Hub (library) to use in back-end for loading model from public models hub or from path</li></ul>|
| **data**      | The data to be used for evaluation. A dictionary providing flexibility and options for data sources. It should include the following keys: <ul><li>data_source (mandatory): The source of the data.</li><li>subset (optional): The subset of the data.</li><li>feature_column (optional): The column containing the features.</li><li>target_column (optional): The column containing the target labels.</li><li>split (optional): The data split to be used.</li><li>source (optional): Set to 'huggingface' when loading Hugging Face dataset.</li></ul> |
| **config**    | Configuration for the tests to be performed, specified in the form of a YAML file. |


<br/>
<br/>

# Real-World Project Workflows

In this section, we dive into complete workflows for using the model testing module in real-world project settings.

## Robustness Testing

In this example, we will be testing a model's robustness. We will be applying 2 tests: add_typo and lowercase. The real-world project workflow of the model robustness testing and fixing in this case goes as follows:

1. Train NER model on original CoNLL training set

2. Test NER model robustness on CoNLL test set

3. Augment CoNLL training set based on test results

4. Train new NER model on augmented CoNLL training set

5. Test new NER model robustness on the CoNLL test set from step 2

6. Compare robustness of new NER model against original NER model

#### Load Train and Test CoNLL

In [3]:
# Load test CoNLL
!wget https://raw.githubusercontent.com/JohnSnowLabs/langtest/main/langtest/data/conll/sample.conll

# Load train CoNLL
!wget https://raw.githubusercontent.com/JohnSnowLabs/langtest/main/demo/data/conll03.conll

--2023-11-30 13:43:59--  https://raw.githubusercontent.com/JohnSnowLabs/langtest/main/langtest/data/conll/sample.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50519 (49K) [text/plain]
Saving to: ‘sample.conll.1’

sample.conll.1      100%[===================>]  49.33K  --.-KB/s    in 0.04s   

2023-11-30 13:44:00 (1.10 MB/s) - ‘sample.conll.1’ saved [50519/50519]

--2023-11-30 13:44:00--  https://raw.githubusercontent.com/JohnSnowLabs/langtest/main/demo/data/conll03.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8274

#### Step 1: Train NER Model

In [4]:
from johnsnowlabs import nlp

In [21]:
ner_model = nlp.load('bert train.ner').fit(dataset_path="/content/conll03.conll")


small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
[OK!]


#### Step 2: Test NER Model Robustness  

In [22]:
harness = Harness(task="ner", model={"model": ner_model, "hub": "johnsnowlabs"}, data={"data_source":"sample.conll"})

Test Configuration : 
 {
 "tests": {
  "defaults": {
   "min_pass_rate": 1.0
  },
  "robustness": {
   "add_typo": {
    "min_pass_rate": 0.7
   },
   "american_to_british": {
    "min_pass_rate": 0.7
   }
  },
  "accuracy": {
   "min_micro_f1_score": {
    "min_score": 0.7
   }
  },
  "bias": {
   "replace_to_female_pronouns": {
    "min_pass_rate": 0.7
   },
   "replace_to_low_income_country": {
    "min_pass_rate": 0.7
   }
  },
  "fairness": {
   "min_gender_f1_score": {
    "min_score": 0.6
   }
  },
  "representation": {
   "min_label_representation_count": {
    "min_count": 50
   }
  }
 }
}


In [23]:
harness.configure({
    'tests': {
        'defaults': {'min_pass_rate': 0.65},

        'robustness': {
            'add_typo': {'min_pass_rate': 0.73},
            'lowercase':{'min_pass_rate': 0.65},
        }
    }
})

{'tests': {'defaults': {'min_pass_rate': 0.65},
  'robustness': {'add_typo': {'min_pass_rate': 0.73},
   'lowercase': {'min_pass_rate': 0.65}}}}

Here we have configured the harness to perform two robustness tests (add_typo and lowercase) and defined the minimum pass rate for each test.


#### Generating the test cases.




In [24]:
harness.generate()


Generating testcases...: 100%|██████████| 1/1 [00:00<00:00, 5184.55it/s]
[W010] - Test 'add_typo': 15 samples removed out of 226
[W010] - Test 'lowercase': 3 samples removed out of 226



harness.generate() method automatically generates the test cases (based on the provided configuration)

In [25]:
harness.testcases()

,category,test_type,original,test_case
0,robustness,add_typo,"SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRI...","SOCCER - JAPAN GET LUCOY WIN , CHINA IN SURPRI..."
1,robustness,add_typo,Nadim Ladki,Nadim Ladoi
2,robustness,add_typo,"AL-AIN , United Arab Emirates 1996-12-06","AL-AIN , Unitev Arab Emirates 1996-12-06"
3,robustness,add_typo,Japan began the defence of their Asian Cup tit...,Japan began the defence of their Asian Cup tit...
4,robustness,add_typo,But China saw their luck desert them in the se...,But China saw their luck desert them in the se...
...,...,...,...,...
429,robustness,lowercase,Portuguesa 1 Atletico Mineiro 0,portuguesa 1 atletico mineiro 0
430,robustness,lowercase,CRICKET - LARA ENDURES ANOTHER MISERABLE DAY .,cricket - lara endures another miserable day .
431,robustness,lowercase,Robert Galvin,robert galvin
432,robustness,lowercase,MELBOURNE 1996-12-06,melbourne 1996-12-06


harness.testcases() method gives the produced test cases in form of a pandas data frame.

#### Saving test configurations, data, test cases

In [26]:
harness.save("saved_test_configurations")

#### Running the tests

In [27]:
harness.run()

Running testcases... : 100%|██████████| 434/434 [00:51<00:00,  8.35it/s]


Called after harness.generate() and is to used to run all the tests.  Returns a pass/fail flag for each test.

In [28]:
harness.generated_results()

,category,test_type,original,test_case,expected_result,actual_result,pass
0,robustness,add_typo,"SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRI...","SOCCER - JAPAN GET LUCOY WIN , CHINA IN SURPRI...","japan: LOC, lucky: LOC, china: LOC","japan: LOC, lucoy: PER, china: LOC",False
1,robustness,add_typo,Nadim Ladki,Nadim Ladoi,nadim ladki: PER,nadim ladoi: PER,True
2,robustness,add_typo,"AL-AIN , United Arab Emirates 1996-12-06","AL-AIN , Unitev Arab Emirates 1996-12-06","al-ain: LOC, united arab emirates: LOC","al-ain: LOC, unitev: PER, arab emirates: LOC",False
3,robustness,add_typo,Japan began the defence of their Asian Cup tit...,Japan began the defence of their Asian Cup tit...,"japan: LOC, asian cup: MISC, syria: LOC","japan: LOC, asian cup: MISC, lucuy: PER, syria...",True
4,robustness,add_typo,But China saw their luck desert them in the se...,But China saw their luck desert them in the se...,"china: LOC, uzbekistan: LOC","china: LOC, yzbekistan: LOC",True
...,...,...,...,...,...,...,...
429,robustness,lowercase,Portuguesa 1 Atletico Mineiro 0,portuguesa 1 atletico mineiro 0,"portuguesa: ORG, atletico mineiro: ORG","portuguesa: ORG, atletico mineiro: ORG",True
430,robustness,lowercase,CRICKET - LARA ENDURES ANOTHER MISERABLE DAY .,cricket - lara endures another miserable day .,lara: PER,lara: PER,True
431,robustness,lowercase,Robert Galvin,robert galvin,robert galvin: PER,robert galvin: PER,True
432,robustness,lowercase,MELBOURNE 1996-12-06,melbourne 1996-12-06,melbourne: LOC,melbourne: LOC,True


This method returns the generated results in the form of a pandas dataframe, which provides a convenient and easy-to-use format for working with the test results. You can use this method to quickly identify the test cases that failed and to determine where fixes are needed.

#### Report of the tests

In [29]:
harness.report()

,category,test_type,fail_count,pass_count,pass_rate,minimum_pass_rate,pass
0,robustness,add_typo,56,155,73%,73%,True
1,robustness,lowercase,0,223,100%,65%,True


It summarizes the results giving information about pass and fail counts and overall test pass/fail flag.

#### Step 3: Augment CoNLL Training Set Based on Robustness Test Results

Templatic Augmentation is a technique that allows you to generate new training data by applying a set of predefined templates to the original training data. The templates are designed to introduce noise into the training data in a way that simulates real-world conditions. The augmentation process is controlled by a configuration file that specifies the augmentation templates to be used and the proportion of the training data to be augmented. The augmentation process is performed by the augment() method of the **Harness** class.

**Augumentation with templates**

Templatic augmentation is controlled by templates to be used with training data to be augmented. The augmentation process is performed by the augment() method of the **Harness** class.

```
templates = ["The {ORG} company is located in {LOC}"]

```


The `.augment()` function takes the following parameters:

- `training_data` (dict): (Required) Specifies the source of the original training data. It should be a dictionary containing the necessary information about the dataset.
- `save_data_path` (str): (Required) Name of the file to store the augmented data. The augmented dataset will be saved in this file.
- `templates` (list): List of templates(string) or conll file to be used for augmentation.
- `generate_templates` (bool): if set to True, generates sample templates from given ones.
- `show_templates` (bool): if set to True, displays the used templates.

In [30]:
data_kwargs = {
      "data_source" : "conll03.conll",
       }

import openai
openai.api_key = "YOUR OPENAI KEY"
harness.augment(
    training_data=data_kwargs,
    save_data_path='augmented_conll03.conll',
    templates=["The {ORG} company is located in {LOC}"],
    generate_templates = True,
    show_templates = True,
    )

The {ORG} company is located in {LOC}
'The {ORG} organization is based in {LOC}',
 '{ORG} is headquartered in {LOC}',
 '{LOC} is the home of {ORG}',
 '{ORG} is situated in {LOC}',
 '{LOC} is where {ORG} is located',
 '{ORG} is found in {LOC}',
 '{LOC} is the location of {ORG}',
 '{ORG} is based in {LOC}',
 '{LOC} is the home of the {ORG} company',
 '{ORG} is situated in the city of {LOC}'


Essentially it applies perturbations to the input data based on the recommendations from the harness reports. Then this augmented_dataset is used to retrain the original model so as to make the model more robust and improve its performance.

In [15]:
!head -n 20 augmented_conll03.conll



The -X- -X- O
Dinamo -X- -X- B-ORG
company -X- -X- O
is -X- -X- O
located -X- -X- O
in -X- -X- O
Yugoslavia -X- -X- B-LOC

The -X- -X- O
Red -X- -X- B-ORG
Star -X- -X- I-ORG
company -X- -X- O
is -X- -X- O
located -X- -X- O
in -X- -X- O
Ghana -X- -X- B-LOC

The -X- -X- O


#### Step 4: Train New NER Model on Augmented CoNLL

In [31]:
augmented_ner_model = nlp.load('bert train.ner').fit(dataset_path= "augmented_conll03.conll")

small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
[OK!]


#### Load saved test configurations, data

In [32]:
harness = Harness.load("saved_test_configurations",
                       model= {"model":augmented_ner_model,"hub":"johnsnowlabs"}, 
                       task="ner",                       
                       load_testcases=True)

Test Configuration : 
 {
 "tests": {
  "defaults": {
   "min_pass_rate": 0.65
  },
  "robustness": {
   "add_typo": {
    "min_pass_rate": 0.73
   },
   "lowercase": {
    "min_pass_rate": 0.65
   }
  }
 }
}



Generating testcases...: 100%|██████████| 1/1 [00:00<00:00, 5127.51it/s]
[W010] - Test 'add_typo': 11 samples removed out of 226
[W010] - Test 'lowercase': 3 samples removed out of 226



#### Step 5: Test New NER Model Robustness

In [33]:
harness.run()

Running testcases... : 100%|██████████| 438/438 [00:50<00:00,  8.63it/s]


In [34]:
harness.generated_results()

,category,test_type,original,test_case,expected_result,actual_result,pass
0,robustness,add_typo,"SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRI...","SOCCER - JAPAN GET LUCKY WIN , CHINA IN WURPRI...","soccer - japan get lucky win , china in surpri...","soccer - japan get lucky win , china in wurpri...",True
1,robustness,add_typo,Nadim Ladki,Nadum Ladki,nadim ladki: ORG,nadum ladki: ORG,True
2,robustness,add_typo,"AL-AIN , United Arab Emirates 1996-12-06","AL-AIG , United Arab Emirates 1996-12-06","al-ain , united arab: ORG, emirates: LOC, 1996...","al-aig , united arab: ORG, emirates: LOC, 1996...",True
3,robustness,add_typo,Japan began the defence of their Asian Cup tit...,Japan began the defence of their Asian Cup tit...,japan began the defence of their asian cup tit...,japan began the defence of their asian cup tit...,True
4,robustness,add_typo,But China saw their luck desert them in the se...,But China saw their luck desert them in the se...,but china saw their luck desert them in the se...,but china saw their luck desert them in the se...,True
...,...,...,...,...,...,...,...
433,robustness,lowercase,Portuguesa 1 Atletico Mineiro 0,portuguesa 1 atletico mineiro 0,portuguesa 1 atletico mineiro 0: ORG,portuguesa 1 atletico mineiro 0: ORG,True
434,robustness,lowercase,CRICKET - LARA ENDURES ANOTHER MISERABLE DAY .,cricket - lara endures another miserable day .,cricket - lara endures another miserable: ORG,cricket - lara endures another miserable: ORG,True
435,robustness,lowercase,Robert Galvin,robert galvin,robert galvin: ORG,robert galvin: ORG,True
436,robustness,lowercase,MELBOURNE 1996-12-06,melbourne 1996-12-06,"melbourne: LOC, 1996-12-06: ORG","melbourne: LOC, 1996-12-06: ORG",True


In [35]:
harness.report()

,category,test_type,fail_count,pass_count,pass_rate,minimum_pass_rate,pass
0,robustness,add_typo,39,176,82%,73%,True
1,robustness,lowercase,0,223,100%,65%,True
